In [2]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

In [3]:
# make sure pandas is version 1.0 or higher
# make sure networkx is verion 2.4 or higher
print(pd.__version__)
print(nx.__version__)

1.5.3
2.8.4


In [4]:
from ema_workbench import (
    Model,
    Policy,
    ema_logging,
    SequentialEvaluator,
    MultiprocessingEvaluator,
)
from dike_model_function import DikeNetwork  # @UnresolvedImport
from problem_formulation import get_model_for_problem_formulation, sum_over, sum_over_time



In [5]:
ema_logging.log_to_stderr(ema_logging.INFO)

# choose problem formulation number, between 0-5
# each problem formulation has its own list of outcomes
dike_model, planning_steps = get_model_for_problem_formulation(3)

In [6]:
# enlisting uncertainties, their types (RealParameter/IntegerParameter/CategoricalParameter), lower boundary, and upper boundary
import copy

for unc in dike_model.uncertainties:
    print(repr(unc))

uncertainties = copy.deepcopy(dike_model.uncertainties)

CategoricalParameter('discount rate 0', [0, 1, 2, 3])
CategoricalParameter('discount rate 1', [0, 1, 2, 3])
CategoricalParameter('discount rate 2', [0, 1, 2, 3])
CategoricalParameter('A.1_Brate', [0, 1, 2])
CategoricalParameter('A.2_Brate', [0, 1, 2])
CategoricalParameter('A.3_Brate', [0, 1, 2])
CategoricalParameter('A.4_Brate', [0, 1, 2])
CategoricalParameter('A.5_Brate', [0, 1, 2])


In [7]:
# enlisting policy levers, their types (RealParameter/IntegerParameter), lower boundary, and upper boundary
for policy in dike_model.levers:
    print(repr(policy))

levers = copy.deepcopy(dike_model.levers)

In [8]:
# enlisting outcomes
for outcome in dike_model.outcomes:
    print(repr(outcome))

ScalarOutcome('A.1 Total Costs', variable_name=('A.1_Expected Annual Damage', 'A.1_Dike Investment Costs'), function=<function sum_over at 0x7fa9899563a0>)
ScalarOutcome('A.1_Expected Number of Deaths', variable_name=('A.1_Expected Number of Deaths',), function=<function sum_over at 0x7fa9899563a0>)
ScalarOutcome('A.2 Total Costs', variable_name=('A.2_Expected Annual Damage', 'A.2_Dike Investment Costs'), function=<function sum_over at 0x7fa9899563a0>)
ScalarOutcome('A.2_Expected Number of Deaths', variable_name=('A.2_Expected Number of Deaths',), function=<function sum_over at 0x7fa9899563a0>)
ScalarOutcome('A.3 Total Costs', variable_name=('A.3_Expected Annual Damage', 'A.3_Dike Investment Costs'), function=<function sum_over at 0x7fa9899563a0>)
ScalarOutcome('A.3_Expected Number of Deaths', variable_name=('A.3_Expected Number of Deaths',), function=<function sum_over at 0x7fa9899563a0>)
ScalarOutcome('A.4 Total Costs', variable_name=('A.4_Expected Annual Damage', 'A.4_Dike Investmen

In [9]:
# running the model through EMA workbench
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(scenarios=50, policies=4)

[MainProcess/INFO] pool started with 10 workers
[MainProcess/INFO] performing 50 scenarios * 4 policies * 1 model(s) = 200 experiments
100%|████████████████████████████████████████| 200/200 [00:07<00:00, 26.28it/s]
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


In [10]:
# observing the simulation runs
experiments, outcomes = results
print(outcomes.keys())
experiments

dict_keys(['A.1 Total Costs', 'A.1_Expected Number of Deaths', 'A.2 Total Costs', 'A.2_Expected Number of Deaths', 'A.3 Total Costs', 'A.3_Expected Number of Deaths', 'A.4 Total Costs', 'A.4_Expected Number of Deaths', 'A.5 Total Costs', 'A.5_Expected Number of Deaths', 'RfR Total Costs', 'Expected Evacuation Costs'])


,A.0_ID flood wave shape,A.1_Bmax,A.1_Brate,A.1_pfail,A.2_Bmax,A.2_Brate,A.2_pfail,A.3_Bmax,A.3_Brate,A.3_pfail,...,A.4_DikeIncrease 0,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,EWS_DaysToThreat,scenario,policy,model
0,116,227.829061,1.0,0.314884,255.915828,10.0,0.381302,332.466979,1.0,0.506031,...,8,9,5,2,7,8,4,4,0,dikesnet
1,34,100.721426,1.5,0.068656,328.615777,10.0,0.771175,187.615352,10.0,0.427465,...,8,9,5,2,7,8,4,5,0,dikesnet
2,61,174.205178,10.0,0.975663,146.384389,10.0,0.987620,217.343138,1.5,0.741285,...,8,9,5,2,7,8,4,6,0,dikesnet
3,10,342.820451,1.5,0.540132,190.070145,1.0,0.133810,121.845622,10.0,0.928837,...,8,9,5,2,7,8,4,7,0,dikesnet
4,36,321.678004,10.0,0.489932,83.931216,10.0,0.964011,234.911333,1.5,0.227290,...,8,9,5,2,7,8,4,8,0,dikesnet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,82,119.749477,1.0,0.916466,164.833288,1.5,0.953422,254.169810,1.5,0.242690,...,1,4,9,4,5,3,0,49,3,dikesnet
196,28,185.586497,1.5,0.689695,63.227588,10.0,0.829309,206.696269,1.0,0.045706,...,1,4,9,4,5,3,0,50,3,dikesnet
197,30,263.186941,10.0,0.156592,185.149856,1.0,0.620980,148.642777,10.0,0.800902,...,1,4,9,4,5,3,0,51,3,dikesnet
198,122,286.052437,1.0,0.465671,249.901672,10.0,0.240660,243.964772,1.0,0.185966,...,1,4,9,4,5,3,0,52,3,dikesnet


In [11]:
# only works because we have scalar outcomes
pd.DataFrame(outcomes)

,A.1 Total Costs,A.1_Expected Number of Deaths,A.2 Total Costs,A.2_Expected Number of Deaths,A.3 Total Costs,A.3_Expected Number of Deaths,A.4 Total Costs,A.4_Expected Number of Deaths,A.5 Total Costs,A.5_Expected Number of Deaths,RfR Total Costs,Expected Evacuation Costs
0,1.449562e+08,0.0,2.081644e+08,0.000847,1.249149e+08,0.00000,5.944456e+07,0.000000,1.404200e+08,0.000000,1.000800e+09,470.626153
1,1.449562e+08,0.0,2.009914e+08,0.000242,1.249149e+08,0.00000,5.944456e+07,0.000000,1.595549e+08,0.001566,1.000800e+09,1188.991533
2,1.449562e+08,0.0,1.982604e+08,0.000000,1.249149e+08,0.00000,5.944456e+07,0.000000,1.404200e+08,0.000000,1.000800e+09,0.000000
3,1.449562e+08,0.0,2.376081e+08,0.005198,1.249149e+08,0.00000,5.944456e+07,0.000000,1.404200e+08,0.000000,1.000800e+09,3107.452957
4,1.449562e+08,0.0,1.982604e+08,0.000000,1.288139e+08,0.00082,5.944456e+07,0.000000,1.911794e+08,0.005186,1.000800e+09,4020.589346
...,...,...,...,...,...,...,...,...,...,...,...,...
195,2.045780e+08,0.0,3.000417e+08,0.000000,1.218614e+08,0.00000,3.781059e+07,0.000000,1.163985e+08,0.000000,1.251600e+09,0.000000
196,2.045780e+08,0.0,3.000417e+08,0.000000,1.218614e+08,0.00000,3.902108e+07,0.000868,1.163985e+08,0.000000,1.251600e+09,0.000000
197,2.045780e+08,0.0,3.000417e+08,0.000000,1.218614e+08,0.00000,3.781059e+07,0.000000,1.163985e+08,0.000000,1.251600e+09,0.000000
198,2.045780e+08,0.0,3.000417e+08,0.000000,1.218614e+08,0.00000,3.781059e+07,0.000000,1.163985e+08,0.000000,1.251600e+09,0.000000


In [12]:
# defining specific policies
# for example, policy 1 is about extra protection in upper boundary
# policy 2 is about extra protection in lower boundary
# policy 3 is extra protection in random locations


def get_do_nothing_dict():
    return {l.name: 0 for l in dike_model.levers}


policies = [
    Policy(
        "policy 1",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 1, "0_RfR 1": 1, "0_RfR 2": 1, "A.1_DikeIncrease 0": 5}
        )
    ),
    Policy(
        "policy 2",
        **dict(
            get_do_nothing_dict(),
            **{"4_RfR 0": 1, "4_RfR 1": 1, "4_RfR 2": 1, "A.5_DikeIncrease 0": 5}
        )
    ),
    Policy(
        "policy 3",
        **dict(
            get_do_nothing_dict(),
            **{"1_RfR 0": 1, "2_RfR 1": 1, "3_RfR 2": 1, "A.3_DikeIncrease 0": 5}
        )
    ),
]

In [13]:
# pass the policies list to EMA workbench experiment runs
n_scenarios = 100
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(n_scenarios, policies)

[MainProcess/INFO] pool started with 10 workers
[MainProcess/INFO] performing 100 scenarios * 3 policies * 1 model(s) = 300 experiments
100%|████████████████████████████████████████| 300/300 [00:12<00:00, 24.96it/s]
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


In [14]:
experiments, outcomes = results

In [15]:
# only works because we have scalar outcomes
pd.DataFrame(outcomes)

,A.1 Total Costs,A.1_Expected Number of Deaths,A.2 Total Costs,A.2_Expected Number of Deaths,A.3 Total Costs,A.3_Expected Number of Deaths,A.4 Total Costs,A.4_Expected Number of Deaths,A.5 Total Costs,A.5_Expected Number of Deaths,RfR Total Costs,Expected Evacuation Costs
0,5.397251e+07,0.000000,1.456509e+08,0.140958,1.140604e+09,2.037104,6.270157e+06,0.002970,3.532720e+08,0.317249,253800000.0,0.0
1,5.397251e+07,0.000000,9.280847e+07,0.078009,2.169010e+07,0.034686,1.016735e+06,0.000461,0.000000e+00,0.000000,253800000.0,0.0
2,5.397251e+07,0.000000,1.054994e+08,0.093604,0.000000e+00,0.000000,1.517131e+07,0.006695,8.875536e+08,0.728919,253800000.0,0.0
3,5.397251e+07,0.000000,2.988322e+08,0.370025,8.400418e+08,1.974631,0.000000e+00,0.000000,0.000000e+00,0.000000,253800000.0,0.0
4,5.397251e+07,0.000000,4.766690e+06,0.006138,6.334425e+08,1.421049,1.302479e+07,0.007597,0.000000e+00,0.000000,253800000.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
295,6.449472e+08,0.424723,0.000000e+00,0.000000,2.879840e+07,0.000000,9.763206e+07,0.042576,0.000000e+00,0.000000,369700000.0,0.0
296,0.000000e+00,0.000000,1.832255e+08,0.164428,2.879840e+07,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,369700000.0,0.0
297,1.118402e+08,0.064348,8.564205e+08,0.644776,2.879840e+07,0.000000,7.329400e+07,0.027965,0.000000e+00,0.000000,369700000.0,0.0
298,3.704246e+08,0.324836,2.042977e+08,0.248160,2.879840e+07,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,369700000.0,0.0
